# 1. Installazione ed Autenticazione

In [ ]:
# Installiamo la libreria di Google per Fogli
!pip install gspread

# Importiamo le librerie necessarie
import gspread
import pandas as pd
from google.colab import auth

# Autentichiamo l'utente
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

print("✅ Autenticazione completata.")

# 2. Connessione e Lettura Dati

In [ ]:
# Apriamo il Google Sheet per NOME.
# Assicurati che il nome sia ESATTO.
try:
    ss = gc.open("FantaStep")
    print("✅ File 'Fanta Step' aperto con successo.")

    # Leggiamo i fogli di input
    db_ws = ss.worksheet("DB_Giocatori")
    voti_ws = ss.worksheet("Input_Voti")
    form_ws = ss.worksheet("Form_Responses")

    # Trasformiamo i fogli in DataFrame Pandas
    # get_all_records() legge i dati usando la prima riga come intestazione
    df_db = pd.DataFrame(db_ws.get_all_records())
    df_voti = pd.DataFrame(voti_ws.get_all_records())
    df_form = pd.DataFrame(form_ws.get_all_records())

    print("DB Giocatori Caricato:", len(df_db), "giocatori")
    print("Input Voti Caricato:", len(df_voti), "voti")
    print("Formazioni Ricevute:", len(df_form), "formazioni")

except Exception as e:
    print(f"🛑 ERRORE: Impossibile aprire il file o i fogli. Controlla i nomi.")
    print(e)

In [ ]:
# --- CELLA DI DEBUG ---
# E' stata necessaria per vedere i nomi esatti delle tabelle di Form (che non sceglieva in modo corretto)

if 'df_form' in locals():
  print("✅ Nomi esatti delle colonne lette da 'Form_Responses':")
  print(list(df_form.columns))
else:
  print("🛑 Errore: Esegui prima la Cella 2 per caricare 'df_form'.")

# 3. Preparazione Dati

In [ ]:
# Pulizia e Calcolo Voti
print("Inizio preparazione dati...")

# Lista delle colonne bonus/malus (devono corrispondere al foglio!)
colonne_bm = [
    'Goal_Fatto', 'Goal_Subito', 'Falli', 'Giallo', 'Rosso',
    'Squadra_Vince', 'Squadra_Perde', 'Porta_Inviolata', 'MVP',
    'Rigore_Parato', 'Rigore_Sbagliato', 'Autogoal'
]

# Convertiamo le colonne B/M in numeri (se vuote, diventano 0)
for col in colonne_bm:
    if col in df_voti:
        df_voti[col] = pd.to_numeric(df_voti[col], errors='coerce').fillna(0)
    else:
        print(f"Attenzione: Colonna '{col}' non trovata in 'Input_Voti'")

# Calcoliamo il punteggio finale per ogni giocatore
# (Se un giocatore non ha voti, riceverà 0, non 6)
df_voti['Punteggio_Finale'] = 6 + \
    (df_voti.get('Goal_Fatto', 0) * 3) - \
    (df_voti.get('Goal_Subito', 0) * 1) - \
    (df_voti.get('Falli', 0) * 0.5) - \
    (df_voti.get('Giallo', 0) * 1) - \
    (df_voti.get('Rosso', 0) * 2) + \
    (df_voti.get('Squadra_Vince', 0) * 0.5) - \
    (df_voti.get('Squadra_Perde', 0) * 0.5) + \
    (df_voti.get('Porta_Inviolata', 0) * 1) + \
    (df_voti.get('MVP', 0) * 1) + \
    (df_voti.get('Rigore_Parato', 0) * 3) - \
    (df_voti.get('Rigore_Sbagliato', 0) * 3) - \
    (df_voti.get('Autogoal', 0) * 3)

# Creazione Mappe (Dizionari)
# Convertiamo 'Costo' in numero per sicurezza
df_db['Costo'] = pd.to_numeric(df_db['Costo'], errors='coerce').fillna(0)

# Mappa Nome -> Costo
# Esempio: {'Gigi': 10, 'Giangiovanni': 15}
prezzi_map = df_db.set_index('Nome_Giocatore')['Costo'].to_dict()

# Mappa Nome -> Punteggio_Finale
# Esempio: {'Gigi': 6.5, 'Giangiovanni': 9.0}
punti_map = df_voti.set_index('Nome_Giocatore')['Punteggio_Finale'].to_dict()

print("✅ Mappe prezzi e punti create.")

# 4. Elaborazione Formazioni


In [ ]:
# ⚠️ CONFIGURAZIONE OBBLIGATORIA ⚠️
COLONNE_MAP = {
    'email': 'Indirizzo email',
    'nome': 'Nome',
    'cognome': 'Cognome',
    'nickname': 'Nickname Squadra',
    'portiere': 'Scelta Portiere',
    'difensori': 'Scelta Difensori',
    'centrocampisti': 'Scelta Centrocampisti',
    'attaccante': 'Scelta Attaccante'
}

# Imposta il tuo budget
BUDGET_MASSIMO = 100 # ⚠️ Modifica questo con il tuo budget!

# Lista per salvare i risultati finali
risultati_giornata = []

# Funzione per pulire il nome. "Giovannino (10)" -> "Giovannino"
def pulisci_nome(nome_raw):
    if not isinstance(nome_raw, str):
        return ""
    return nome_raw.split(' (')[0].strip()

print(f"Inizio elaborazione di {len(df_form)} formazioni...")
print(f"Budget massimo impostato: {BUDGET_MASSIMO}")

for index, riga in df_form.iterrows():

    squadra_raw = []
    costo_totale = 0
    punteggio_totale = 0
    status = "VALIDA"
    squadra_pulita = []

    # Identificativo utente
    try:
        # La chiave univoca è l'email
        email_utente = riga[COLONNE_MAP['email']]
        nome_utente = f"{riga[COLONNE_MAP['nome']]} {riga[COLONNE_MAP['cognome']]}"
        nickname_squadra = riga[COLONNE_MAP['nickname']]

        # Gestiamo nickname vuoti
        if not nickname_squadra or pd.isna(nickname_squadra):
            nickname_squadra = nome_utente # Fallback sul nome utente

    except KeyError as e:
        print(f"ERRORE CRITICO: Impossibile leggere colonna {e}. Fermo elaborazione riga.")
        continue # Salta questa riga
    except Exception as e:
        email_utente = f"Errore_Email_{index}"
        nome_utente = f"Errore_Nome_{index}"
        nickname_squadra = "Errore_Nickname"
        print(f"Errore lettura metadati utente: {e}")

    try:
        # 1. RACCOLTA E PULIZIA GIOCATORI
        p_raw = riga[COLONNE_MAP['portiere']]
        a_raw = riga[COLONNE_MAP['attaccante']]
        squadra_raw.extend([p_raw, a_raw])

        dif_raw_list = riga[COLONNE_MAP['difensori']].split(',')
        cen_raw_list = riga[COLONNE_MAP['centrocampisti']].split(',')

        squadra_raw.extend([d.strip() for d in dif_raw_list])
        squadra_raw.extend([c.strip() for c in cen_raw_list])

        squadra_pulita = [pulisci_nome(g) for g in squadra_raw if pulisci_nome(g)]

        # 2. VALIDAZIONE
        if len(squadra_pulita) != 7:
            status = f"INVALIDA ({len(squadra_pulita)} giocatori trovati, non 7)"
        else:
            for nome in squadra_pulita:
                costo_giocatore = prezzi_map.get(nome)
                punti_giocatore = punti_map.get(nome, 6)

                if costo_giocatore is None:
                    status = f"INVALIDA (Giocatore '{nome}' non trovato nel DB)"
                    costo_totale = 0
                    break

                costo_totale += costo_giocatore
                punteggio_totale += punti_giocatore

            if status == "VALIDA" and costo_totale > BUDGET_MASSIMO:
                status = f"INVALIDA (Budget sforato: {costo_totale})"

        # 3. Assegnazione Punteggio
        if status != "VALIDA":
            punteggio_totale = 0

        risultati_giornata.append({
            'Email': email_utente, # <-- NUOVA CHIAVE
            'Utente': nome_utente, # Nome e Cognome
            'Nickname': nickname_squadra, # Nickname
            'Punteggio': round(punteggio_totale, 2),
            'Costo': costo_totale,
            'Status': status,
            'Squadra': ", ".join(squadra_pulita)
        })

    except Exception as e:
        risultati_giornata.append({
            'Email': email_utente,
            'Utente': nome_utente,
            'Nickname': nickname_squadra,
            'Punteggio': 0, 'Costo': 0,
            'Status': f"ERRORE LETTURA (Cella vuota? Dettaglio: {e})",
            'Squadra': ""
        })

print("✅ Elaborazione terminata.")

# Convertiamo i risultati in un DataFrame
df_risultati = pd.DataFrame(risultati_giornata)
# Impostiamo l'Email come indice per i controlli (anche se non strettamente nec.)
df_risultati.set_index('Email', inplace=True)

print("\n--- RISULTATI GIORNATA (Top 10) ---")
print(df_risultati.sort_values(by="Punteggio", ascending=False).head(10))

print("\n--- FORMAZIONI CON ERRORI O INVALIDE ---")
print(df_risultati[df_risultati['Status'] != 'VALIDA'])

# Riportiamo l'Email a colonna per la Cella successiva
df_risultati.reset_index(inplace=True)

# 5. Aggiornamento Classifica Storica

In [ ]:
# Chiedi il nome (prefisso) per la nuova giornata
nome_giornata_prefix = input("Inserisci il PREFISSO per questa giornata (es. G1, G2, etc.): ")

if not nome_giornata_prefix:
    print("🛑 Prefisso nullo. Operazione annullata.")
else:
    print(f"Inizio aggiornamento classifica con prefisso '{nome_giornata_prefix}'...")
    try:
        # Prepara i punteggi della giornata corrente
        cols_da_tenere = ['Email', 'Utente', 'Nickname', 'Punteggio', 'Costo', 'Status', 'Squadra']
        df_giornata_nuova = df_risultati[cols_da_tenere].copy()

        df_giornata_nuova['Email'] = df_giornata_nuova['Email'].astype(str).str.strip().str.lower()

        colonne_rinominate = {
            'Punteggio': f"{nome_giornata_prefix}_Punteggio",
            'Costo': f"{nome_giornata_prefix}_Costo",
            'Status': f"{nome_giornata_prefix}_Status",
            'Squadra': f"{nome_giornata_prefix}_Squadra"
        }
        df_giornata_nuova.rename(columns=colonne_rinominate, inplace=True)
        df_giornata_nuova.set_index('Email', inplace=True)

        # Leggi la classifica storica esistente (COME TESTO)
        classifica_ws = ss.worksheet("Classifica")

        # --- FIX: Torniamo a get_all_values() per leggere tutto come TESTO ---
        sheet_data_values = classifica_ws.get_all_values()

        df_storica = pd.DataFrame()

        if len(sheet_data_values) > 1:
            print("Letta classifica storica esistente.")
            header = sheet_data_values[0]
            data = sheet_data_values[1:]
            df_storica = pd.DataFrame(data, columns=header)
            # --- FINE FIX ---

            if 'Email' not in df_storica.columns:
                 raise ValueError("La classifica storica non ha la colonna 'Email'.")

            df_storica['Email'] = df_storica['Email'].astype(str).str.strip().str.lower()
            df_storica = df_storica.set_index('Email')

            colonne_orfane = ['Punteggio', 'Costo', 'Status', 'Squadra']
            df_storica = df_storica.drop(columns=[col for col in colonne_orfane if col in df_storica.columns], errors='ignore')

            colonna_check = f"{nome_giornata_prefix}_Punteggio"
            if colonna_check in df_storica.columns:
                print(f"⚠️ ATTENZIONE: Le colonne con prefisso '{nome_giornata_prefix}' esistono già.")
                sovrascrivi = input("Vuoi sovrascrivere i dati? (sì/no): ")
                if sovrascrivi.lower() != 'sì':
                    raise Exception("Operazione annullata dall'utente.")
                else:
                    cols_da_rimuovere = [col for col in df_storica.columns if col.startswith(nome_giornata_prefix)]
                    df_storica = df_storica.drop(columns=cols_da_rimuovere)
        else:
            print("Foglio 'Classifica' vuoto. Questa è la prima giornata.")

        # Unisci la classifica vecchia e i punteggi nuovi
        df_aggiornata = df_storica.join(df_giornata_nuova, how='outer')

        # --- Pulisci e Ricalcola il Totale (PRIMA DI TUTTO) ---
        # Questo blocco ora viene eseguito sul DataFrame "stabile", prima che venga "avvelenato"
        if 'Punteggio_Totale' in df_aggiornata.columns:
            df_aggiornata = df_aggiornata.drop(columns=['Punteggio_Totale'])

        colonne_punteggi = [col for col in df_aggiornata.columns if col.endswith('_Punteggio')]
        print(f"Calcolo totale basato su: {colonne_punteggi}")

        punteggio_totale_calcolato = pd.Series(0.0, index=df_aggiornata.index)

        for col in colonne_punteggi:
            # --- FIX: Sostituisci la virgola E converti ---
            col_stringa_pulita = df_aggiornata[col].astype(str).str.replace(',', '.')
            col_numerica = pd.to_numeric(col_stringa_pulita, errors='coerce').fillna(0)
            # --- FINE FIX ---

            df_aggiornata[col] = col_numerica
            punteggio_totale_calcolato = punteggio_totale_calcolato.add(col_numerica, fill_value=0)

        df_aggiornata['Punteggio_Totale'] = punteggio_totale_calcolato

        # --- Logica di Overwrite Nomi (DOPO CHE I CALCOLI SONO FINITI) ---
        # Ora questa logica può "avvelenare" il DataFrame, ma non ci interessa più
        df_aggiornata['Utente_Display'] = df_aggiornata['Utente'].fillna(df_aggiornata.get('Utente_Display', ''))
        df_aggiornata['Nickname_Display'] = df_aggiornata['Nickname'].fillna(df_aggiornata.get('Nickname_Display', ''))

        df_aggiornata = df_aggiornata.drop(columns=['Utente', 'Nickname'], errors='ignore')

        # 5.7. Formatta e Ordina
        df_aggiornata['Punteggio_Totale'] = df_aggiornata['Punteggio_Totale'].round(2)
        df_aggiornata.sort_values(by='Punteggio_Totale', ascending=False, inplace=True)

        cols_finali = ['Punteggio_Totale', 'Utente_Display', 'Nickname_Display']
        altre_colonne = sorted([col for col in df_aggiornata.columns if col not in cols_finali])
        cols_finali.extend(altre_colonne)

        df_aggiornata = df_aggiornata[cols_finali]

        colonne_testo = [col for col in df_aggiornata.columns if col.endswith('_Status') or col.endswith('_Squadra')]
        df_aggiornata[colonne_testo] = df_aggiornata[colonne_testo].fillna('N/D')

        df_aggiornata = df_aggiornata.fillna(0)

        df_aggiornata.reset_index(inplace=True)
        df_aggiornata.rename(columns={'index': 'Email'}, inplace=True)

        # 5.8. Scrivi sul foglio Google
        print("Scrittura della classifica aggiornata in corso...")
        classifica_ws.clear()

        colonne_output = ['Email', 'Punteggio_Totale', 'Utente_Display', 'Nickname_Display'] + altre_colonne
        df_aggiornata = df_aggiornata[colonne_output]

        output_data = [df_aggiornata.columns.values.tolist()] + df_aggiornata.values.tolist()

        classifica_ws.update(range_name='A1', values=output_data, value_input_option='USER_ENTERED')

        print(f"\n✅ Classifica aggiornata con successo con i dati di '{nome_giornata_prefix}'!")
        print(f"Link: {ss.url}")

    except Exception as e:
        print(f"\n🛑 ERRORE durante l'aggiornamento della classifica:")
        print(e)

# 6. Genera file HTML

In [ ]:
from google.colab import files
import datetime

# Assicurati che df_aggiornata esista e sia aggiornata
if 'df_aggiornata' not in locals():
    print("🛑 ERRORE: Esegui prima la Cella 5 per calcolare 'df_aggiornata'.")
else:
    print("Inizio generazione file HTML...")

    # Ordiniamo per sicurezza (anche se dovrebbe già esserlo)
    df_html = df_aggiornata.sort_values(by='Punteggio_Totale', ascending=False)

    # Prendiamo la data e ora attuale per l'aggiornamento
    now = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

    # --- Inizio Creazione Stringa HTML ---

    # CSS per uno stile pulito (puoi personalizzarlo)
    html_css = """
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif; margin: 0; padding: 20px; background-color: #f4f7f6; }
        h1 { color: #333; text-align: center; }
        .container { max-width: 800px; margin: 20px auto; background-color: #fff; border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.05); }
        ol { list-style-type: none; padding: 0; margin: 0; }
        li {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 15px 20px;
            border-bottom: 1px solid #eee;
            font-size: 1.1em;
        }
        li:last-child { border-bottom: none; }
        .position { font-weight: bold; color: #888; min-width: 30px; }
        .details { flex-grow: 1; margin-left: 15px; }
        .nickname { font-weight: 600; color: #111; }
        .utente { font-size: 0.9em; color: #777; }
        .punteggio { font-weight: bold; font-size: 1.2em; color: #0056b3; }
        footer { text-align: center; margin-top: 20px; font-size: 0.9em; color: #888; }
    </style>
    """

    # HTML Body
    html_body = f"""
    <div class="container">
        <ol>
    """

    # Aggiungi ogni riga della classifica
    for index, riga in df_html.iterrows():
        posizione = index + 1
        nickname = riga.get('Nickname_Display', 'N/D')
        utente = riga.get('Utente_Display', 'N/D')
        punteggio = riga.get('Punteggio_Totale', 0)

        html_body += f"""
            <li>
                <span class="position">{posizione}.</span>
                <div class="details">
                    <div class="nickname">{nickname}</div>
                    <div class="utente">({utente})</div>
                </div>
                <span class="punteggio">{punteggio}</span>
            </li>
        """

    html_body += """
        </ol>
    </div>
    """

    # Combina tutto nel file finale
    html_content = f"""
    <!DOCTYPE html>
    <html lang="it">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Classifica Fanta Step</title>
        {html_css}
    </head>
    <body>
        <h1>Classifica Generale Fanta Step</h1>
        {html_body}
        <footer>
            Ultimo aggiornamento: {now}
        </footer>
    </body>
    </html>
    """

    # --- Fine Creazione Stringa HTML ---

    # Salva il file
    file_name = "index.html"
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(html_content)

    print(f"✅ File '{file_name}' creato con successo.")

    # Fai partire il download automatico
    files.download(file_name)